# 数据结构的内置方法

这一节介绍常用的pandas数据结构内置方法。很重要的一节。

创建本节要用到的数据结构。

In [1]:
import numpy as np
import pandas as pd

In [2]:
index  = pd.date_range('1/1/2000', periods=8)
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=['A', 'B', 'C'])
wp = pd.Panel(np.random.randn(2,5,4), items=['Item1', 'Item2'], major_axis=pd.date_range('1/1/2000',periods=5),
             minor_axis=['A', 'B', 'C', 'D'])

## Head() Tail()

想要预览Series或DataFrame对象，可以使用head()和tail()方法。默认显示5行数据，你也可以自己设置显示的行数。

In [3]:
long_series = pd.Series(np.random.randn(1000))

In [4]:
long_series.head()

0    0.023058
1   -0.074953
2    0.561164
3    1.955282
4    0.737479
dtype: float64

In [5]:
long_series.tail(3)

997   -1.318555
998    0.047255
999    0.218609
dtype: float64

## 属性和 ndarray

pandas对象有很多属性，你可以通过这些属性访问数据。

* **shape**: 显示对象的维度，同ndarray
* 坐标label
* * **Series**: index
* * **DataFrame**: index(行)和columns
* * **Panel**:  items, major_axis and minor_axis

可以通过属性进行安全赋值。

In [6]:
df[:2]

,A,B,C
2000-01-01,1.173233,-0.147631,-1.312819
2000-01-02,-1.266111,-1.125446,0.578458


In [7]:
df.columns = [x.lower() for x in df.columns] #将列名重置为小写

In [8]:
df

,a,b,c
2000-01-01,1.173233,-0.147631,-1.312819
2000-01-02,-1.266111,-1.125446,0.578458
2000-01-03,-1.595159,-0.654289,-1.256511
2000-01-04,-0.907272,-0.054247,-0.596867
2000-01-05,-1.259856,3.229433,1.971463
2000-01-06,0.108100,0.019998,1.320694
2000-01-07,1.719391,-0.506610,1.039305
2000-01-08,1.107930,-0.191327,0.547800


只想得到对象中的数据而忽略index和columns，使用values属性就可以

In [9]:
s.values

array([ 0.61338428, -0.31247701, -0.54252781,  0.25162093, -0.55944074])

In [10]:
df.values

array([[ 1.17323268, -0.14763078, -1.312819  ],
       [-1.26611102, -1.12544635,  0.57845752],
       [-1.59515867, -0.6542893 , -1.25651072],
       [-0.90727235, -0.05424687, -0.59686748],
       [-1.25985608,  3.229433  ,  1.97146342],
       [ 0.10810032,  0.01999776,  1.32069354],
       [ 1.71939091, -0.50660987,  1.03930483],
       [ 1.1079299 , -0.19132729,  0.54780008]])

In [11]:
type(df.values)

numpy.ndarray

In [12]:
wp.values

array([[[-1.54942434,  0.29147405,  1.04168054, -0.63775426],
        [-0.68954438, -0.30728114,  0.93123234,  1.04587453],
        [ 0.52590821, -0.87797709, -1.4053343 , -1.26458272],
        [ 0.07689833, -0.62319929, -0.03040175,  0.28789213],
        [ 0.26621579,  0.2508008 , -0.13140246,  1.46492144]],

       [[ 1.06565079, -0.09858943, -0.39071567, -0.67769326],
        [ 0.07779078, -0.16233238, -0.42585241, -0.28715586],
        [ 2.41442006,  0.05856983,  0.33624126, -1.19499484],
        [ 1.22066297, -1.40060425,  0.53552757,  0.79119371],
        [-0.54880785,  0.43709614,  0.42075427, -0.58245336]]])

如果DataFrame或Panel对象的数据类型相同(比如都是 int64)，修改object.values相当于直接修改原对象的值。如果数据类型不相同，则根本不能对values属性返回值进行赋值。

**注意**:

如果对象内数据类型不同，values返回的ndarray的dtype将是能够兼容所有数据类型的类型。比如，有的列数据是int，有的列数据是float，.values返回的ndarray的dtype将是float。

## 加速的操作

pandas从0.11.0版本开始使用numexpr库对二值数值类型操作加速，用bottleneck库对布尔操作加速。

加速效果对大数据尤其明显。

这里有一个速度的简单对比，使用100,000行\* 100列的DataFrame:

![](https://ooo.0o0.ooo/2016/04/14/570f692c2e8a1.png)

所以，在安装pandas后也要顺便安装numexpr, bottleneck。

## 灵活的二元运算

在所有的pandas对象之间的二元运算中，大家最感兴趣的一般是下面两个：
* 高维数据结构（比如DataFrame）和低维数据结构（比如Series）之间计算时的广播(broadcasting)行为
* 计算时有缺失值



### 广播

DataFrame对象内置add(), sub(), mul(), div()以及radd(), rsub(),...等方法来进行二元操作。

使用这些方法时，你可以通过axis参数来决定广播时匹配index还是columns:

In [13]:
df = pd.DataFrame({'one' : pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
                       'two' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
                       'three' : pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})

In [14]:
df

,one,three,two
a,0.930595,NaN,0.693425
b,-1.452673,0.021665,-0.406850
c,0.284763,0.236374,-0.568421
d,NaN,-0.660751,0.675628


In [15]:
row = df.ix[1]
row

one     -1.452673
three    0.021665
two     -0.406850
Name: b, dtype: float64

In [16]:
column = df['two']

In [17]:
column

a    0.693425
b   -0.406850
c   -0.568421
d    0.675628
Name: two, dtype: float64

In [18]:
df.sub(row, axis='columns')

,one,three,two
a,2.383268,NaN,1.100275
b,0.000000,0.000000,0.000000
c,1.737436,0.214709,-0.161571
d,NaN,-0.682416,1.082478


In [19]:
df.sub(row, axis=1)

,one,three,two
a,2.383268,NaN,1.100275
b,0.000000,0.000000,0.000000
c,1.737436,0.214709,-0.161571
d,NaN,-0.682416,1.082478


In [20]:
df.sub(row, axis='index')

,one,three,two
a,NaN,NaN,NaN
b,NaN,NaN,NaN
c,NaN,NaN,NaN
d,NaN,NaN,NaN
one,NaN,NaN,NaN
three,NaN,NaN,NaN
two,NaN,NaN,NaN


In [21]:
df.sub(row, axis=0)

,one,three,two
a,NaN,NaN,NaN
b,NaN,NaN,NaN
c,NaN,NaN,NaN
d,NaN,NaN,NaN
one,NaN,NaN,NaN
three,NaN,NaN,NaN
two,NaN,NaN,NaN


### 填充缺失值

在Series和DataFrame中，算术运算方法(比如add())有一个fill_value参数，含义很明显，计算前用一个值来代替缺失值，然后再参与运算。注意，如果参与运算的两个object同一位置(同行同列)都是NaN，fill_value不起作用，计算结果还是NaN。

看例子：

In [22]:
df

,one,three,two
a,0.930595,NaN,0.693425
b,-1.452673,0.021665,-0.406850
c,0.284763,0.236374,-0.568421
d,NaN,-0.660751,0.675628


In [23]:
df2 = pd.DataFrame({'one' : pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
                       'two' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
                       'three' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd'])})

In [24]:
df2

,one,three,two
a,-0.213464,0.984343,0.832706
b,-0.592199,-0.131885,-0.599882
c,-0.092918,-1.011125,-1.246759
d,NaN,-0.658080,1.424326


In [25]:
df + df2

,one,three,two
a,0.717132,NaN,1.526131
b,-2.044872,-0.110220,-1.006732
c,0.191845,-0.774751,-1.815180
d,NaN,-1.318831,2.099954


In [26]:
 df.add(df2, fill_value=0) #注意['a', 'three']不是NaN

,one,three,two
a,0.717132,0.984343,1.526131
b,-2.044872,-0.110220,-1.006732
c,0.191845,-0.774751,-1.815180
d,NaN,-1.318831,2.099954


### 灵活的比较操作

pandas引入了二元比较运算方法：eq, ne, lt, gt, le。

In [27]:
df.gt(df2)

,one,three,two
a,True,False,False
b,False,True,True
c,True,True,True
d,False,False,False


In [28]:
df2.ne(df)

,one,three,two
a,True,True,True
b,True,True,True
c,True,True,True
d,True,True,True


意思操作返回一个和输入对象同类型的对象，值类型为bool，返回结果可以用于检索。

### 布尔降维 Boolean Reductions

pandas提供了三个方法(any(), all(), bool())和一个empty属性来对布尔结果进行降维。

In [29]:
df>0

,one,three,two
a,True,False,True
b,False,True,False
c,True,True,False
d,False,False,True


In [30]:
(df>0).all() #与操作

one      False
three    False
two      False
dtype: bool

In [31]:
(df > 0).any()#或操作

one      True
three    True
two      True
dtype: bool

同样可以对降维后的结果再进行降维。

In [32]:
(df > 0).any().any()

True

使用empty属性检测一个pandas对象是否为空。

In [33]:
df.empty

False

In [34]:
pd.DataFrame(columns=list('ABC')).empty

True

对于只含有一个元素的pandas对象，对其进行布尔检测，使用bool():

In [35]:
pd.Series([True]).bool()

True

In [36]:
pd.Series([False]).bool()

False

In [37]:
pd.DataFrame([[True]]).bool()

True

In [38]:
pd.DataFrame([[False]]).bool()

False

### 比较对象是否相等

一个问题通常有多种解法。一个最简单的例子：df+df和df\*2。为了检测两个计算结果是否相等，你可能想到：(df+df == df\*2).all()，然而，这样计算得到的结果是False：

In [39]:
df + df == df*2

,one,three,two
a,True,False,True
b,True,True,True
c,True,True,True
d,False,True,True


In [40]:
(df+df == df*2).all()

one      False
three    False
two       True
dtype: bool

为什么df + df == df\*2 返回的结果含有False？因为NaN和NaN比较厚结果为False！

In [41]:
np.nan == np.nan

False

还好pandas提供了equals()方法解决上面NaN之间不想等的问题。

In [42]:
(df+df).equals(df*2)

True

**注意**：

在使用equals()方法进行比较时，两个对象如果数据不一致必为False。甚至是如果label的顺序不一致也返回False。

In [47]:
df1 = pd.DataFrame({'c':['f',0,np.nan]})
df1

,c
0,f
1,0
2,NaN


In [48]:
df2 = pd.DataFrame({'c':[np.nan, 0, 'f']}, index=[2,1,0])
df2

,c
2,NaN
1,0
0,f


In [49]:
df1.equals(df2)

False

In [50]:
df1.equals(df2.sort_index()) #对df2的索引排序，然后再比较

True

### 不同类型的对象之间 逐元素比较

你可以直接对pandas对象和一个常量值进行逐元素比较：

In [51]:
pd.Series(['foo', 'bar', 'baz']) == 'foo'

0     True
1    False
2    False
dtype: bool

In [52]:
pd.Index(['foo', 'bar', 'baz']) == 'foo'

array([ True, False, False], dtype=bool)

不同类型的对象(比如pandas数据结构、numpy数组)之间进行逐元素的比较也是没有问题的，**前提是两个对象的shape要相同**。

In [53]:
pd.Series(['foo', 'bar', 'baz']) == pd.Index(['foo', 'bar', 'qux'])

0     True
1     True
2    False
dtype: bool

In [54]:
pd.Series(['foo', 'bar', 'baz']) == np.array(['foo', 'bar', 'qux'])

0     True
1     True
2    False
dtype: bool

In [55]:
pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo', 'bar']) #长度不相同

ValueError: Can only compare identically-labeled Series objects

但要知道不同shape的numpy数组之间是可以直接比较的！因为广播！即使无法广播，也不会Error而是返回False。

In [56]:
np.array([1,2,3]) == np.array([2])

array([False,  True, False], dtype=bool)

In [57]:
np.array([1, 2, 3]) == np.array([1, 2])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


False

###  combine_first()

看一下例子：

In [58]:
df1 = pd.DataFrame({'A' : [1., np.nan, 3., 5., np.nan],
                    'B' : [np.nan, 2., 3., np.nan, 6.]})
df1

,A,B
0,1.0,NaN
1,NaN,2.0
2,3.0,3.0
3,5.0,NaN
4,NaN,6.0


In [59]:
df2 = pd.DataFrame({'A' : [5., 2., 4., np.nan, 3., 7.],
                        'B' : [np.nan, np.nan, 3., 4., 6., 8.]})
df2

,A,B
0,5.0,NaN
1,2.0,NaN
2,4.0,3.0
3,NaN,4.0
4,3.0,6.0
5,7.0,8.0


In [60]:
df1.combine_first(df2)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


解释：

对于df1中NaN的元素，用df2中对应位置的元素替换！

### DataFrame.combine()

DataFrame.combine()方法接收一个DF对象和一个combiner方法。

In [61]:
combiner = lambda x,y: np.where(pd.isnull(x), y,x)

In [62]:
df1.combine(df2, combiner)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


## 统计相关 的方法

Series, DataFrame和Panel内置了许多计算统计相关指标的方法。这些方法大致分为两类：
* 返回低维结果，比如sum(),mean(),quantile()
* 返回同原对象同样大小的对象，比如cumsum(), cumprod()

总体来说，这些方法接收一个坐标轴参数:
* Series不需要坐标轴参数
* DataFrame 默认axis=0(index), axis=1(columns)
* Panel 默认axis=1(major), axis=0(items), axis=2(minor)

In [63]:
df

,one,three,two
a,0.930595,NaN,0.693425
b,-1.452673,0.021665,-0.406850
c,0.284763,0.236374,-0.568421
d,NaN,-0.660751,0.675628


In [64]:
df.mean() #axis=0, 计算每一列的平均值

one     -0.079105
three   -0.134238
two      0.098445
dtype: float64

In [65]:
df.mean(1) #计算每一行的平均值

a    0.812010
b   -0.612619
c   -0.015761
d    0.007438
dtype: float64

所有的这些方法都有skipna参数，含义是计算过程中是否剔除缺失值，skipna默认值为True。

In [66]:
df.sum(0, skipna=False)

one           NaN
three         NaN
two      0.393782
dtype: float64

In [67]:
df.sum(axis=1, skipna=True)

a    1.624021
b   -1.837858
c   -0.047284
d    0.014876
dtype: float64

**这些函数可以参与算术和广播运算。**

比如：

In [68]:
ts_stand = (df-df.mean())/df.std()

In [69]:
ts_stand.std()

one      1.0
three    1.0
two      1.0
dtype: float64

In [70]:
xs_stand = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)

In [71]:
xs_stand.std(1)

a    1.0
b    1.0
c    1.0
d    1.0
dtype: float64

注意cumsum() cumprod()方法 保留NA值的位置。

In [72]:
df.cumsum()

,one,three,two
a,0.930595,NaN,0.693425
b,-0.522077,0.021665,0.286575
c,-0.237314,0.258039,-0.281846
d,NaN,-0.402713,0.393782


下面列出常用的方法及其描述。提醒每一个方法都有一个level参数用于具有层次索引的对象。




|    方法     | 描述  | 
| ------------- |:-------------:| 
| count      | 沿着坐标轴统计非空的行数| 
| sum      | 沿着坐标轴取加和|   
| mean | 沿着坐标轴求均值|    
|mad|沿着坐标轴计算平均绝对偏差|
|median|沿着坐标轴计算中位数|
|min|沿着坐标轴取最小值|
|max|沿着坐标轴取最大值|
|mode|沿着坐标轴取众数|
|abs|计算每一个值的绝对值|
|prod|沿着坐标轴求乘积|
|std|沿着坐标轴计算标准差|
|var|沿着坐标轴计算无偏方差|
|sem|沿着坐标轴计算标准差|
|skew|沿着坐标轴计算样本偏斜|
|kurt|沿着坐标轴计算样本峰度|
|quantile|沿着坐标轴计算样本分位数，单位%|
|cumsum|沿着坐标轴计算累加和|
|cumprod|沿着坐标轴计算累积乘|
|cummax|沿着坐标轴计算累计最大|
|cummin|沿着坐标轴计算累计最小|


**Note**：所有需要沿着坐标轴计算的方法，默认axis=0，即将方法应用到每一列数据上。

Series还有一个nunique()方法返回非空数值 组成的集合的大小。

In [73]:
series = pd.Series(np.random.randn(500))
series[20:500]=np.nan

In [74]:
series[10:20]=5

In [75]:
series.nunique()

11

In [76]:
series

0      0.466212
1     -1.572517
2     -1.789153
3     -0.649339
4      0.874092
5     -0.591584
6      0.596416
7     -0.257149
8      0.822657
9     -0.340937
10     5.000000
11     5.000000
12     5.000000
13     5.000000
14     5.000000
15     5.000000
16     5.000000
17     5.000000
18     5.000000
19     5.000000
20          NaN
21          NaN
22          NaN
23          NaN
24          NaN
25          NaN
26          NaN
27          NaN
28          NaN
29          NaN
         ...   
470         NaN
471         NaN
472         NaN
473         NaN
474         NaN
475         NaN
476         NaN
477         NaN
478         NaN
479         NaN
480         NaN
481         NaN
482         NaN
483         NaN
484         NaN
485         NaN
486         NaN
487         NaN
488         NaN
489         NaN
490         NaN
491         NaN
492         NaN
493         NaN
494         NaN
495         NaN
496         NaN
497         NaN
498         NaN
499         NaN
dtype: float64

### descrieb()， 数据摘要

describe()方法非常有用，它计算数据的各种常用统计指标(比如平均值、标准差等)，计算时不包括NA。拿到数据首先要有大概的了解，使用describe()方法就对了。

In [77]:
series = pd.Series(np.random.randn(1000))
series[::2]=np.nan

In [78]:
series.describe()

count    500.000000
mean       0.008860
std        0.955782
min       -2.968920
25%       -0.686382
50%        0.018842
75%        0.635701
max        2.574386
dtype: float64

In [79]:
frame = pd.DataFrame(np.random.randn(1000, 5), columns=['a', 'b', 'c', 'd', 'e'])

In [80]:
frame.ix[::2]=np.nan

In [81]:
frame.describe()

,a,b,c,d,e
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.005870,-0.056518,0.007845,0.045578,-0.001946
std,0.984246,1.045085,1.012088,1.038450,1.048679
min,-2.752012,-3.414155,-2.518559,-2.884816,-2.871252
25%,-0.645610,-0.769178,-0.730541,-0.630003,-0.712117
50%,-0.008707,-0.077536,-0.009839,0.013494,0.031158
75%,0.631591,0.631172,0.675323,0.762697,0.688557
max,2.689364,2.846510,3.668186,3.027345,2.877428


默认describe()只包含25%, 50%, 75%, 也可以通过percentiles参数进行指定。

In [82]:
series.describe(percentiles=[.05, .25, .75, .95])

count    500.000000
mean       0.008860
std        0.955782
min       -2.968920
5%        -1.508605
25%       -0.686382
50%        0.018842
75%        0.635701
95%        1.658959
max        2.574386
dtype: float64

如果Series内数据是非数值类型，describe()也能给出一定的统计结果

In [83]:
s = pd.Series(['a', 'a', 'b', 'b', 'a', 'a', np.nan, 'c', 'd', 'a'])

In [84]:
s.describe()

count     9
unique    4
top       a
freq      5
dtype: object

如果DataFrame对象有的列是数值类型，有的列不是数值类型，describe()仅对数值类型的列进行计算。

In [85]:
frame = pd.DataFrame({'a':['Yes', 'Yes', 'NO', 'No'], 'b':range(4)})

In [86]:
frame.describe()

,b
count,4.000000
mean,1.500000
std,1.290994
min,0.000000
25%,0.750000
50%,1.500000
75%,2.250000
max,3.000000


如果非要知道非数值列的统计指标呢？describe提供了include参数，取值范围{'object', 'number', 'all'}。

看一下例子, 注意'object'和'number'都是在列表中，而'all'不需要放在列表中：

In [87]:
frame.describe(include=['object']) #只对非数值列进行统计计算

,a
count,4
unique,3
top,Yes
freq,2


In [88]:
frame.describe(include=['number'])

,b
count,4.000000
mean,1.500000
std,1.290994
min,0.000000
25%,0.750000
50%,1.500000
75%,2.250000
max,3.000000


In [89]:
frame.describe(include='all')#'all'不是列表

,a,b
count,4,4.000000
unique,3,NaN
top,Yes,NaN
freq,2,NaN
mean,NaN,1.500000
std,NaN,1.290994
min,NaN,0.000000
25%,NaN,0.750000
50%,NaN,1.500000
75%,NaN,2.250000


### 最大/最小值对应的index

idxmin(), idxmax()返回最小值和最大值对应的index。

In [97]:
s1 = pd.Series(np.random.randn(5))
s1

0    0.366003
1   -0.169499
2   -0.624169
3   -0.307402
4   -0.599977
dtype: float64

In [98]:
s1.idxmin(), s1.idxmax() #最小值：-0.296405, 最大值：1.735420

(2, 0)

In [99]:
df1 = pd.DataFrame(np.random.randn(5,3), columns=list('ABC'))
df1

,A,B,C
0,-0.996110,1.576533,-0.950779
1,0.140296,0.499200,1.119654
2,-1.187044,0.449324,0.830870
3,1.661936,-0.241894,0.086323
4,-1.748093,-0.496080,0.972968


In [100]:
df1.idxmin(axis=0)

A    4
B    4
C    0
dtype: int64

In [101]:
df1.idxmax(axis=1)

0    B
1    C
2    C
3    A
4    C
dtype: object

如果多个数值都是最大值或最小值，idxmax() idxmin()返回最大值、最小值第一次出现对应的索引值

In [102]:
df3 = pd.DataFrame([2, 1, 1, 3, np.nan], columns=['A'], index=list('edcba'))
df3

,A
e,2.0
d,1.0
c,1.0
b,3.0
a,NaN


In [103]:
df3['A'].idxmin()

'd'

**实际上，idxmin和idxmax就是NumPy中的argmin和argmax**。

###   value_counts() 数值计数

value_counts()计算一维度数据结构的直方图。

In [104]:
data = np.random.randint(0, 7, size=50)
data

array([1, 5, 4, 6, 0, 5, 6, 6, 2, 0, 6, 5, 6, 6, 3, 2, 1, 3, 3, 4, 3, 6, 5,
       4, 3, 1, 3, 1, 0, 5, 2, 1, 2, 4, 4, 4, 1, 2, 4, 0, 6, 2, 5, 5, 2, 4,
       4, 3, 5, 3])

In [105]:
s = pd.Series(data)

In [106]:
s.value_counts()

4    9
6    8
5    8
3    8
2    7
1    6
0    4
dtype: int64

In [107]:
pd.value_counts(data) #也是全局方法

4    9
6    8
5    8
3    8
2    7
1    6
0    4
dtype: int64

虽然前面介绍过mode()方法了，看两个例子吧：

In [108]:
s5 = pd.Series([1,1,3,3,3,5,5,7,7,7])

In [109]:
s5.mode()

0    3
1    7
dtype: int64

In [110]:
df5 = pd.DataFrame({"A": np.random.randint(0, 7, size=50),
                    "B": np.random.randint(-10, 15, size=50)})

In [111]:
df5

,A,B
0,5,-9
1,4,-6
2,2,11
3,3,11
4,4,6
5,0,3
6,5,2
7,5,4
8,3,-4
9,2,-5


In [112]:
df5.mode()

,A,B
0,4,7


### 区间离散化

cut() qcut()方法可以对连续数据进行离散化

In [113]:
arr = np.random.randn(20)

In [114]:
factor = pd.cut(arr, 4)

In [115]:
factor

[(-0.597, 0.476], (-1.674, -0.597], (-1.674, -0.597], (-1.674, -0.597], (1.548, 2.621], ..., (-0.597, 0.476], (0.476, 1.548], (-0.597, 0.476], (-1.674, -0.597], (-1.674, -0.597]]
Length: 20
Categories (4, object): [(-1.674, -0.597] < (-0.597, 0.476] < (0.476, 1.548] < (1.548, 2.621]]

In [116]:
factor = pd.cut(arr, [-5, -1, 0, 1, 5]) #输入 离散区间

In [117]:
factor

[(-1, 0], (-5, -1], (-5, -1], (-1, 0], (1, 5], ..., (-1, 0], (0, 1], (-1, 0], (-1, 0], (-1, 0]]
Length: 20
Categories (4, object): [(-5, -1] < (-1, 0] < (0, 1] < (1, 5]]

qcut()方法计算样本的分位数，比如我们可以将正态分布的数据 进行四分位数离散化：

In [118]:
arr = np.random.randn(30)

In [119]:
factor = pd.qcut(arr, [0, .25, .5, .75, 1])

In [120]:
factor

[(0.216, 0.851], [-2.228, -0.68], (0.851, 2.636], [-2.228, -0.68], (-0.68, 0.216], ..., (-0.68, 0.216], (0.216, 0.851], (0.216, 0.851], (-0.68, 0.216], [-2.228, -0.68]]
Length: 30
Categories (4, object): [[-2.228, -0.68] < (-0.68, 0.216] < (0.216, 0.851] < (0.851, 2.636]]

In [121]:
pd.value_counts(factor)

(0.851, 2.636]     8
[-2.228, -0.68]    8
(0.216, 0.851]     7
(-0.68, 0.216]     7
dtype: int64

离散区间也可以用极限定义

In [122]:
arr = np.random.randn(20)

In [123]:
factor = pd.cut(arr, [-np.inf, 0, np.inf])

In [124]:
factor

[(0, inf], (-inf, 0], (-inf, 0], (-inf, 0], (0, inf], ..., (-inf, 0], (0, inf], (0, inf], (0, inf], (0, inf]]
Length: 20
Categories (2, object): [(-inf, 0] < (0, inf]]

## 函数应用 

如果你想用自己写的方法或其他库方法操作pandas对象，你应该知道下面的三种方式。 具体选择哪种方式取决于你是想操作整个DataFrame对象还是DataFrame对象的某几行或某几列，或者逐元素操作。

* **管道 pipe()**
* **基于列或行的函数应用 apply()**
* **对DataFrame对象逐元素计算 applymap()**



### 管道

DataFrame和Series当然能够作为参数传入方法。然而，如果涉及到多个方法的序列调用，推荐使用pipe()。看一下例子：

In [ ]:
#f, g 和h是三个方法，接收DataFrame对象，返回DataFrame对象
f(g(h(df), arg1=1), arg2=2, arg3=3)

上面一行代码推荐用下面的等价写法:

In [ ]:
(df.pipe(h).pipe(g, arg1=1).pipe(f, arg2=2, arg3=3))

注意 f g h三个方法中DataFrame都是作为第一个参数。如果DataFrame作为第二个参数呢？方法是为pipe提供(callable, data_keyword)，pipe会自动调用DataFrame对象。

比如，使用statsmodels处理回归问题，他们的API期望第一个参数是公式，第二个参数是DataFrame对象data。我们使用pipe传递(sm.poisson, 'data'):

![](https://ooo.0o0.ooo/2016/04/16/5712057590c85.png)

pipe灵感来自于Unix中伟大的艺术：管道。pandas中pipe()的实现很简洁，推荐阅读源代码pd.DataFrame.pipe

### 基于行或者列的函数应用

任意函数都可以直接对DataFrame或Panel某一坐标轴进行直接操纵，只需要使用apply()方法即可，同描述性统计方法一样，apply()方法接收axis参数。

In [5]:
df.apply(np.mean)

A   -0.082000
B   -0.039589
C   -0.226022
dtype: float64

In [7]:
df.apply(np.mean, axis=1)

2000-01-01    0.031020
2000-01-02    0.754696
2000-01-03   -0.861633
2000-01-04    0.707869
2000-01-05   -0.191752
2000-01-06    0.329149
2000-01-07   -1.213012
2000-01-08   -0.483304
Freq: D, dtype: float64

In [11]:
df.apply(lambda x: x.max() - x.min())

A    1.991008
B    3.883607
C    3.555682
dtype: float64

In [12]:
df.apply(np.cumsum)

,A,B,C
2000-01-01,-0.142261,-0.060360,0.295683
2000-01-02,0.460755,2.086278,-0.189882
2000-01-03,-0.532777,0.994217,-0.689186
2000-01-04,0.464698,2.055012,-0.623848
2000-01-05,0.784671,1.499974,-0.964040
2000-01-06,0.516696,0.778774,1.012581
2000-01-07,0.193691,-0.958194,-0.566480
2000-01-08,-0.656002,-0.316716,-1.808178


In [13]:
df.apply(np.exp)

,A,B,C
2000-01-01,0.867394,0.941425,1.344044
2000-01-02,1.827624,8.556042,0.615349
2000-01-03,0.370266,0.335524,0.606953
2000-01-04,2.711427,2.888666,1.067520
2000-01-05,1.377091,0.574050,0.711634
2000-01-06,0.764927,0.486169,7.218311
2000-01-07,0.723970,0.176053,0.206169
2000-01-08,0.427546,1.899287,0.288893


灵活运用apply()方法可以统计出数据集的很多特性。比如，假设我们希望从数据中抽取每一列最大值的索引值。

In [21]:
tsdf = pd.DataFrame(np.random.randn(1000, 3), columns=['A', 'B', 'C'],
                   index=pd.date_range('1/1/2000', periods=1000))
tsdf

,A,B,C
2000-01-01,1.508512,-0.337924,-0.322681
2000-01-02,0.546119,-0.569760,-0.475934
2000-01-03,0.479077,0.141438,-1.540489
2000-01-04,-0.829857,-0.823538,1.643723
2000-01-05,0.255760,1.210986,0.931286
2000-01-06,2.070021,1.276612,-0.115412
2000-01-07,-0.473810,0.291923,-1.695630
2000-01-08,0.602344,-1.268760,0.181028
2000-01-09,1.004540,-0.616671,-0.362409
2000-01-10,0.485239,0.971017,-0.087146


In [22]:
tsdf.apply(lambda x:x.idxmax())

A   2001-07-13
B   2002-05-25
C   2001-11-26
dtype: datetime64[ns]

apply()方法当然支持接收其他参数了，比如下面的例子：

In [23]:
def subtract_and_divide(x, sub, divide=1):
    return (x - sub)/divide

In [24]:
df.apply(subtract_and_divide, args=(5,),divide=3)

,A,B,C
2000-01-01,-1.714087,-1.686787,-1.568106
2000-01-02,-1.465661,-0.951121,-1.828522
2000-01-03,-1.997844,-2.030687,-1.833101
2000-01-04,-1.334175,-1.313068,-1.644887
2000-01-05,-1.560009,-1.851679,-1.780064
2000-01-06,-1.755992,-1.907066,-1.007793
2000-01-07,-1.774335,-2.245656,-2.193020
2000-01-08,-1.949898,-1.452840,-2.080566


另一个有用的特性是对DataFrame对象传递Series方法，然后针对DF对象的每一列或每一行执行 Series内置的方法！

In [26]:
df

,A,B,C
2000-01-01,-0.142261,-0.060360,0.295683
2000-01-02,0.603017,2.146638,-0.485565
2000-01-03,-0.993533,-1.092061,-0.499304
2000-01-04,0.997475,1.060795,0.065338
2000-01-05,0.319973,-0.555038,-0.340192
2000-01-06,-0.267975,-0.721199,1.976621
2000-01-07,-0.323005,-1.736969,-1.579061
2000-01-08,-0.849693,0.641479,-1.241698


In [27]:
df.apply(pd.Series.interpolate)

,A,B,C
2000-01-01,-0.142261,-0.060360,0.295683
2000-01-02,0.603017,2.146638,-0.485565
2000-01-03,-0.993533,-1.092061,-0.499304
2000-01-04,0.997475,1.060795,0.065338
2000-01-05,0.319973,-0.555038,-0.340192
2000-01-06,-0.267975,-0.721199,1.976621
2000-01-07,-0.323005,-1.736969,-1.579061
2000-01-08,-0.849693,0.641479,-1.241698


### 对单个元素应用Python方法

既然不是所有的方法都能被向量化(接收NumPy数组，返回另一个数组或者值),但是DataFrame内置的applymap()和Series的map()方法能够接收任意的接收一个值且返回一个值的Python方法。

In [125]:
df4 = pd.DataFrame(np.random.randn(4, 3),index=['a','b','c','d'],columns=['one', 'two', 'three'])
df4

,one,two,three
a,-0.539083,-2.681732,-0.183946
b,-0.824301,1.172048,0.387141
c,-0.251208,-1.129691,0.937757
d,0.554184,0.016947,0.755570


In [126]:
f = lambda x:len(str(x))

In [127]:
df4['one'].map(f)

a    15
b    15
c    15
d    14
Name: one, dtype: int64

In [128]:
df4.applymap(f)

,one,two,three
a,15,14,15
b,15,13,14
c,15,14,14
d,14,15,14


Series.map()还有一个功能是模仿merge(), join()

In [129]:
s = pd.Series(['six', 'seven', 'six', 'seven', 'six'], index=['a', 'b', 'c', 'd', 'e'])

In [130]:
t = pd.Series({'six':6., 'seven':7.})

In [131]:
s

a      six
b    seven
c      six
d    seven
e      six
dtype: object

In [132]:
t

seven    7.0
six      6.0
dtype: float64

In [133]:
s.map(t)

a    6.0
b    7.0
c    6.0
d    7.0
e    6.0
dtype: float64

## reindex和改变label

reindex()是pandas中基本的数据对齐方法。其他所有依赖label对齐的方法基本都要靠reindex()实现。reindex(重新索引)意味着是沿着某条轴转换数据以匹配新设定的label。具体来说，reindex()做了三件事情：
* 对数据进行排序以匹配新的labels
* 如果新label对应的位置没有数据，插入缺失值NA
* 可以指定调用fill填充数据。

下面是一个简单的例子：

In [134]:
s = pd.Series(np.random.randn(5), index=['a','b','c','d','e'])

In [44]:
s

a    0.735912
b   -0.102690
c    2.403671
d   -0.472632
e   -0.735369
dtype: float64

In [45]:
s.reindex(['e', 'b', 'f', 'd'])

e   -0.735369
b   -0.102690
f         NaN
d   -0.472632
dtype: float64

对于DataFrame来说，你可以同时改变列名和索引值。

In [48]:
df

,A,B,C
2000-01-01,-0.142261,-0.060360,0.295683
2000-01-02,0.603017,2.146638,-0.485565
2000-01-03,-0.993533,-1.092061,-0.499304
2000-01-04,0.997475,1.060795,0.065338
2000-01-05,0.319973,-0.555038,-0.340192
2000-01-06,-0.267975,-0.721199,1.976621
2000-01-07,-0.323005,-1.736969,-1.579061
2000-01-08,-0.849693,0.641479,-1.241698


In [49]:
df.reindex(index=['c', 'f', 'b'], columns=['three', 'two', 'one'])

,three,two,one
c,NaN,NaN,NaN
f,NaN,NaN,NaN
b,NaN,NaN,NaN


如果只想改变列或者索引的label，DataFrame也提供了reindex_axis()方法，接收label和axis。

In [50]:
rs = s.reindex(df.index)

In [51]:
rs

2000-01-01   NaN
2000-01-02   NaN
2000-01-03   NaN
2000-01-04   NaN
2000-01-05   NaN
2000-01-06   NaN
2000-01-07   NaN
2000-01-08   NaN
Freq: D, dtype: float64

In [52]:
rs.index is df.index

True

上面一行代码顺便说明了Series的索引和DataFrame的索引是同一类的实例。

### 重新索引来和另一个对象对齐 reindex_like()

你可能想传递一个对象，使得原来对象的label和传入的对象一样，使用reindex_like()即可。

In [3]:
df2 = pd.DataFrame(np.random.randn(3, 2),index=['a','b','c'],columns=['one', 'two'])

In [4]:
df2

,one,two
a,-0.207263,0.618381
b,0.271959,-0.013542
c,0.486730,1.308178


In [56]:
df

,A,B,C
2000-01-01,-0.142261,-0.060360,0.295683
2000-01-02,0.603017,2.146638,-0.485565
2000-01-03,-0.993533,-1.092061,-0.499304
2000-01-04,0.997475,1.060795,0.065338
2000-01-05,0.319973,-0.555038,-0.340192
2000-01-06,-0.267975,-0.721199,1.976621
2000-01-07,-0.323005,-1.736969,-1.579061
2000-01-08,-0.849693,0.641479,-1.241698


In [57]:
df.reindex_like(df2)

,one,two
a,NaN,NaN
b,NaN,NaN
c,NaN,NaN


### 使用align() 是两个对象相互对齐

align()方法是让两个对象同事对齐的最快方法。它含有join参数，
* join='outer': 取得两个对象的索引并集，这也是join的默认值。
* join='left': 使用调用对象的索引值
* join='right'：使用被调用对象的索引值
* join='inner': 使用两个对象的索引交集

align()方法返回一个元组，元素元素是重新索引的Series对象。

In [5]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])


In [9]:
s1 = s[:4]
s2 = s[1:]

In [10]:
s1

a   -1.131265
b    0.267754
c    0.679747
d    0.185136
dtype: float64

In [11]:
s2

b    0.267754
c    0.679747
d    0.185136
e   -0.440251
dtype: float64

In [12]:
s1.align(s2)

(a   -1.131265
 b    0.267754
 c    0.679747
 d    0.185136
 e         NaN
 dtype: float64, a         NaN
 b    0.267754
 c    0.679747
 d    0.185136
 e   -0.440251
 dtype: float64)

In [14]:
s1.align(s2, join='inner') #交集是 'b', 'c', 'd'

(b    0.267754
 c    0.679747
 d    0.185136
 dtype: float64, b    0.267754
 c    0.679747
 d    0.185136
 dtype: float64)

对于DataFrame来说，join方法默认会应用到索引和列名。

In [16]:
df

,A,B,C
2000-01-01,-0.064020,1.089580,-1.007871
2000-01-02,-1.067990,0.075548,-1.326231
2000-01-03,0.813001,0.010123,0.729032
2000-01-04,0.283158,0.252955,-0.196908
2000-01-05,-1.481362,-0.773020,-0.177803
2000-01-06,1.760388,-1.276605,0.583953
2000-01-07,1.197909,0.013248,0.027281
2000-01-08,1.000680,-0.523045,0.821203


In [23]:
df2 = df.iloc[:5,:2]

In [24]:
df2

,A,B
2000-01-01,-0.064020,1.089580
2000-01-02,-1.067990,0.075548
2000-01-03,0.813001,0.010123
2000-01-04,0.283158,0.252955
2000-01-05,-1.481362,-0.773020


In [25]:
df.align(df2, join='inner')

(                   A         B
 2000-01-01 -0.064020  1.089580
 2000-01-02 -1.067990  0.075548
 2000-01-03  0.813001  0.010123
 2000-01-04  0.283158  0.252955
 2000-01-05 -1.481362 -0.773020,                    A         B
 2000-01-01 -0.064020  1.089580
 2000-01-02 -1.067990  0.075548
 2000-01-03  0.813001  0.010123
 2000-01-04  0.283158  0.252955
 2000-01-05 -1.481362 -0.773020)

In [26]:
df.align(df2)

(                   A         B         C
 2000-01-01 -0.064020  1.089580 -1.007871
 2000-01-02 -1.067990  0.075548 -1.326231
 2000-01-03  0.813001  0.010123  0.729032
 2000-01-04  0.283158  0.252955 -0.196908
 2000-01-05 -1.481362 -0.773020 -0.177803
 2000-01-06  1.760388 -1.276605  0.583953
 2000-01-07  1.197909  0.013248  0.027281
 2000-01-08  1.000680 -0.523045  0.821203,                    A         B   C
 2000-01-01 -0.064020  1.089580 NaN
 2000-01-02 -1.067990  0.075548 NaN
 2000-01-03  0.813001  0.010123 NaN
 2000-01-04  0.283158  0.252955 NaN
 2000-01-05 -1.481362 -0.773020 NaN
 2000-01-06       NaN       NaN NaN
 2000-01-07       NaN       NaN NaN
 2000-01-08       NaN       NaN NaN)

align()也含有一个axis参数，指定仅对于某一坐标轴进行对齐。

In [27]:
df.align(df2, join='inner', axis=0)

(                   A         B         C
 2000-01-01 -0.064020  1.089580 -1.007871
 2000-01-02 -1.067990  0.075548 -1.326231
 2000-01-03  0.813001  0.010123  0.729032
 2000-01-04  0.283158  0.252955 -0.196908
 2000-01-05 -1.481362 -0.773020 -0.177803,                    A         B
 2000-01-01 -0.064020  1.089580
 2000-01-02 -1.067990  0.075548
 2000-01-03  0.813001  0.010123
 2000-01-04  0.283158  0.252955
 2000-01-05 -1.481362 -0.773020)

DataFrame.align()同样能接收Series对象，此时axis指的是DataFrame对象的索引或列。

In [28]:
df.align(df2.ix[0], axis=1)

(                   A         B         C
 2000-01-01 -0.064020  1.089580 -1.007871
 2000-01-02 -1.067990  0.075548 -1.326231
 2000-01-03  0.813001  0.010123  0.729032
 2000-01-04  0.283158  0.252955 -0.196908
 2000-01-05 -1.481362 -0.773020 -0.177803
 2000-01-06  1.760388 -1.276605  0.583953
 2000-01-07  1.197909  0.013248  0.027281
 2000-01-08  1.000680 -0.523045  0.821203, A   -0.06402
 B    1.08958
 C        NaN
 Name: 2000-01-01 00:00:00, dtype: float64)

### 重索引时顺便填充数值

reindex()方法还有一个method参数，用于填充数值,method取值如下：
* pad/ffill: 使用**后面的值**填充数值
* bfill/backfill: 使用**前面的值**填充数值
* nearest: 使用最近的索引值进行填充

以Series为例，看一下：

In [30]:
rng = pd.date_range('1/3/2000', periods=8)

In [31]:
ts = pd.Series(np.random.randn(8), index=rng)

In [32]:
ts2 = ts[[0, 3, 6]]

In [33]:
ts

2000-01-03   -0.765910
2000-01-04    0.077131
2000-01-05   -0.714099
2000-01-06   -0.109570
2000-01-07   -0.942098
2000-01-08   -0.242916
2000-01-09   -0.733449
2000-01-10    0.671837
Freq: D, dtype: float64

In [34]:
ts2

2000-01-03   -0.765910
2000-01-06   -0.109570
2000-01-09   -0.733449
dtype: float64

In [35]:
ts2.reindex(ts.index)

2000-01-03   -0.765910
2000-01-04         NaN
2000-01-05         NaN
2000-01-06   -0.109570
2000-01-07         NaN
2000-01-08         NaN
2000-01-09   -0.733449
2000-01-10         NaN
Freq: D, dtype: float64

In [36]:
ts2.reindex(ts.index, method='ffill') #索引小那一行的数值填充NaN

2000-01-03   -0.765910
2000-01-04   -0.765910
2000-01-05   -0.765910
2000-01-06   -0.109570
2000-01-07   -0.109570
2000-01-08   -0.109570
2000-01-09   -0.733449
2000-01-10   -0.733449
Freq: D, dtype: float64

In [38]:
ts2.reindex(ts.index, method='bfill') #索引大的非NaN的数值填充NaN

2000-01-03   -0.765910
2000-01-04   -0.109570
2000-01-05   -0.109570
2000-01-06   -0.109570
2000-01-07   -0.733449
2000-01-08   -0.733449
2000-01-09   -0.733449
2000-01-10         NaN
Freq: D, dtype: float64

In [39]:
ts2.reindex(ts.index, method='nearest')

2000-01-03   -0.765910
2000-01-04   -0.765910
2000-01-05   -0.109570
2000-01-06   -0.109570
2000-01-07   -0.109570
2000-01-08   -0.733449
2000-01-09   -0.733449
2000-01-10   -0.733449
Freq: D, dtype: float64

method参数要求索引必须是有序的：递增或递减。

除了method='nearest',其他method取值也能用fillna()方法实现：

In [40]:
ts2.reindex(ts.index).fillna(method='ffill')

2000-01-03   -0.765910
2000-01-04   -0.765910
2000-01-05   -0.765910
2000-01-06   -0.109570
2000-01-07   -0.109570
2000-01-08   -0.109570
2000-01-09   -0.733449
2000-01-10   -0.733449
Freq: D, dtype: float64

二者的区别是：如果索引不是有序的，reindex()会报错，而fillna()和interpolate()不会检查索引是否有序。

### 重索引时 有条件地填充NaN

limit和tolerance参数会对填充操作进行条件限制，通常限制填充的次数。

In [41]:
ts2.reindex(ts.index, method='ffill', limit=1)

2000-01-03   -0.765910
2000-01-04   -0.765910
2000-01-05         NaN
2000-01-06   -0.109570
2000-01-07   -0.109570
2000-01-08         NaN
2000-01-09   -0.733449
2000-01-10   -0.733449
Freq: D, dtype: float64

In [42]:
ts2

2000-01-03   -0.765910
2000-01-06   -0.109570
2000-01-09   -0.733449
dtype: float64

In [43]:
ts

2000-01-03   -0.765910
2000-01-04    0.077131
2000-01-05   -0.714099
2000-01-06   -0.109570
2000-01-07   -0.942098
2000-01-08   -0.242916
2000-01-09   -0.733449
2000-01-10    0.671837
Freq: D, dtype: float64

In [44]:
ts2.reindex(ts.index, method='ffill', tolerance='1 day')

2000-01-03   -0.765910
2000-01-04   -0.765910
2000-01-05         NaN
2000-01-06   -0.109570
2000-01-07   -0.109570
2000-01-08         NaN
2000-01-09   -0.733449
2000-01-10   -0.733449
Freq: D, dtype: float64

### 移除某些索引值

和reindex()方法很相似的是drop()，用于移除索引的某些取值。

In [52]:
df

,A,B,C
2000-01-01,-0.064020,1.089580,-1.007871
2000-01-02,-1.067990,0.075548,-1.326231
2000-01-03,0.813001,0.010123,0.729032
2000-01-04,0.283158,0.252955,-0.196908
2000-01-05,-1.481362,-0.773020,-0.177803
2000-01-06,1.760388,-1.276605,0.583953
2000-01-07,1.197909,0.013248,0.027281
2000-01-08,1.000680,-0.523045,0.821203


In [53]:
df.drop(['A'], axis=1)

,B,C
2000-01-01,1.089580,-1.007871
2000-01-02,0.075548,-1.326231
2000-01-03,0.010123,0.729032
2000-01-04,0.252955,-0.196908
2000-01-05,-0.773020,-0.177803
2000-01-06,-1.276605,0.583953
2000-01-07,0.013248,0.027281
2000-01-08,-0.523045,0.821203


### 重命名索引值

rename() 方法可以对索引值重新命名，命名方式可以是字典或Series，也可以是任意的方法。

In [56]:
s

a   -1.131265
b    0.267754
c    0.679747
d    0.185136
e   -0.440251
dtype: float64

In [57]:
s.rename(str.upper)

A   -1.131265
B    0.267754
C    0.679747
D    0.185136
E   -0.440251
dtype: float64

唯一的要求是传入的函数调用索引值时必须有一个返回值，如果你传入的是字典或Series，要求是索引值必须是其键值。这点很好理解。

In [59]:
df = pd.DataFrame({'one' : pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
                   'two' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
                    'three' : pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})
df

,one,three,two
a,-0.074436,NaN,-1.720569
b,1.154073,-0.335651,0.238351
c,0.147157,0.848397,0.827290
d,NaN,0.699900,-1.079261


In [60]:
df.rename(columns={'one' : 'foo', 'two' : 'bar'},
          index={'a' : 'apple', 'b' : 'banana', 'd' : 'durian'})

,foo,three,bar
apple,-0.074436,NaN,-1.720569
banana,1.154073,-0.335651,0.238351
c,0.147157,0.848397,0.827290
durian,NaN,0.699900,-1.079261


默认情况下修改的仅仅是副本，如果想对原对象索引值修改，inplace=True.

在0.18.0版本中，rename方法也能修改Series.name

In [61]:
s.rename('sclar-name')

a   -1.131265
b    0.267754
c    0.679747
d    0.185136
e   -0.440251
Name: sclar-name, dtype: float64

## 迭代操作 Iteration

pandas中迭代操作依赖于具体的对象。迭代Series对象时类似迭代数组,产生的是值，迭代DataFrame或Panel对象时类似迭代字典的键值。

一句话，(for i in object)产生:
* **Series**: 值
* **DataFrame**: 列名
* **Panel**: item名

看一下例子吧：

In [62]:
df = pd.DataFrame({'col1' : np.random.randn(3), 'col2' : np.random.randn(3)},
                  index=['a', 'b', 'c'])
df

,col1,col2
a,0.172851,-0.382805
b,-0.066942,0.672660
c,-2.091001,0.510819


In [64]:
for col in df: #产生的是列名
    print col

col1
col2


pandas对象也有类似字典的iteritems()方法来迭代(key, value)。

为了每一行迭代DataFrame对象，有两种方法：
* **iterrows()**: 按照行来迭代(index, Series)。会把每一行转为Series对象。
* **itertuples()**: 按照行来迭代namedtuples. 这种方法比iterrows()快，大多数情况下推荐使用此方法。

**警告**：

迭代pandas对象通常会比较慢。所以尽量避免迭代操作，可以用以下方法替换迭代：
* 数据结构的内置方法，索引或numpy方法等
* apply()
* 使用cython写内循环

**警告**：

**当迭代进行时永远不要有修改操作**。

## iteritems()

类似字典的接口，iteritems()对键值对进行迭代操作：
* Series: (index, scalar value)
* DataFrame: (column, Series)
* Panel: (item, DataFrame)


看一下例子：

In [65]:
for item, frame in df.iteritems():
    print item, frame

col1 a    0.172851
b   -0.066942
c   -2.091001
Name: col1, dtype: float64
col2 a   -0.382805
b    0.672660
c    0.510819
Name: col2, dtype: float64


### iterrows()

iterrows()方法用于迭代DataFrame的每一行，返回的是索引和Series的迭代器，但要注意Series的dtype可能和原来每一行的dtype不同。

In [68]:
for row_index, row in df.iterrows():
    print  row_index, row

a col1    0.172851
col2   -0.382805
Name: a, dtype: float64
b col1   -0.066942
col2    0.672660
Name: b, dtype: float64
c col1   -2.091001
col2    0.510819
Name: c, dtype: float64


### itertuples()

itertuples()方法迭代DataFrame每一行，返回的是namedtuple。因为返回的不是Series，所以会保留DataFrame中值的dtype。

In [70]:
for row in df.itertuples():
    print row

Pandas(Index='a', col1=0.17285076817271622, col2=-0.38280463605747844)
Pandas(Index='b', col1=-0.066941511967637243, col2=0.67266041180134961)
Pandas(Index='c', col1=-2.0910005480409719, col2=0.51081932119946261)


## .dt 访问器

Series对象如果索引是datetime/period,可以用自带的.dt访问器返回日期、小时、分钟。

In [71]:
s = pd.Series(pd.date_range('20160101 09:10:12', periods=4))

In [72]:
s

0   2016-01-01 09:10:12
1   2016-01-02 09:10:12
2   2016-01-03 09:10:12
3   2016-01-04 09:10:12
dtype: datetime64[ns]

In [73]:
s.dt.hour

0    9
1    9
2    9
3    9
dtype: int64

In [74]:
s.dt.second

0    12
1    12
2    12
3    12
dtype: int64

In [75]:
s.dt.day

0    1
1    2
2    3
3    4
dtype: int64

改变时间的格式也很方便，Series.dt.strftime()

In [76]:
s = pd.Series(pd.date_range('20130101', periods=4))

In [77]:
s

0   2013-01-01
1   2013-01-02
2   2013-01-03
3   2013-01-04
dtype: datetime64[ns]

In [78]:
s.dt.strftime('%Y/%m/%d')

0    2013/01/01
1    2013/01/02
2    2013/01/03
3    2013/01/04
dtype: object

 ## 字符串处理方法

Series带有一系列的字符串处理, 默认不对NaN处理。

In [79]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])

In [80]:
s

0       A
1       B
2       C
3    Aaba
4    Baca
5     NaN
6    CABA
7     dog
8     cat
dtype: object

In [81]:
s.str.lower()

0       a
1       b
2       c
3    aaba
4    baca
5     NaN
6    caba
7     dog
8     cat
dtype: object

## 排序

排序方法可以分为两大类： 按照实际的值排序和按照label排序。

### 按照index排序 sort_index()

Series.sort_index(), DataFrame.sort_index(), 参数是ascending, axis。

In [88]:
unsorted_df = df.reindex(index=['a', 'd', 'c', 'b'],
                         columns=['three', 'two', 'one'])
unsorted_df

,three,two,one
a,NaN,NaN,NaN
d,NaN,NaN,NaN
c,NaN,NaN,NaN
b,NaN,NaN,NaN


In [89]:
unsorted_df.sort_index()

,three,two,one
a,NaN,NaN,NaN
b,NaN,NaN,NaN
c,NaN,NaN,NaN
d,NaN,NaN,NaN


In [90]:
unsorted_df.sort_index(ascending=False)

,three,two,one
d,NaN,NaN,NaN
c,NaN,NaN,NaN
b,NaN,NaN,NaN
a,NaN,NaN,NaN


In [91]:
unsorted_df.sort_index(axis=1)

,one,three,two
a,NaN,NaN,NaN
d,NaN,NaN,NaN
c,NaN,NaN,NaN
b,NaN,NaN,NaN


In [92]:
unsorted_df['three'].sort_index() # Series

a   NaN
b   NaN
c   NaN
d   NaN
Name: three, dtype: float64

### 按照值排序

Series.sort_values(), DataFrame.sort_values()用于按照值进行排序，参数有by

In [93]:
df1 = pd.DataFrame({'one':[2,1,1,1],'two':[1,3,2,4],'three':[5,4,3,2]})
df1

,one,three,two
0,2,5,1
1,1,4,3
2,1,3,2
3,1,2,4


In [94]:
df1.sort_values(by='two')

,one,three,two
0,2,5,1
2,1,3,2
1,1,4,3
3,1,2,4


In [95]:
df1.sort_values(by=['one', 'two'])

,one,three,two
2,1,3,2
1,1,4,3
3,1,2,4
0,2,5,1


通过na_position参数处理NA值。

In [96]:
s[2] = np.nan

In [97]:
s.sort_values()

0       A
3    Aaba
1       B
4    Baca
6    CABA
8     cat
7     dog
2     NaN
5     NaN
dtype: object

In [99]:
s.sort_values(na_position='first') #将NA放在前面

2     NaN
5     NaN
0       A
3    Aaba
1       B
4    Baca
6    CABA
8     cat
7     dog
dtype: object

### searchsorted()

Series.searchsorted()类似numpy.ndarray.searchsorted()。 找到元素在排好序后的位置(下标)。

In [3]:
ser = pd.Series([1,2,3])

In [10]:
ser.searchsorted([0, 3]) #元素0下标是0， 元素3下标是2。注意不同元素之间是独立的，所以元素3的位置是2而不是插入0后的3.

array([0, 3], dtype=int64)

In [11]:
ser.searchsorted([0, 4]) 

array([0, 3], dtype=int64)

In [6]:
ser.searchsorted([1, 3], side='right')

array([1, 3], dtype=int64)

In [7]:
ser.searchsorted([1, 3], side='left')

array([0, 2], dtype=int64)

In [8]:
ser = pd.Series([3, 1, 2])

In [9]:
ser.searchsorted([0, 3], sorter=np.argsort(ser))

array([0, 2], dtype=int64)

### 最小/最大值

Series有nsmallest() nlargest()方法能够返回最小或最大的n个值。如果Series对象很大，这两种方法会比先排序后使用head()方法快很多。

In [12]:
s = pd.Series(np.random.permutation(10))
s

0    6
1    3
2    8
3    7
4    5
5    2
6    4
7    0
8    1
9    9
dtype: int32

In [13]:
s.sort_values()

7    0
8    1
5    2
1    3
6    4
4    5
0    6
3    7
2    8
9    9
dtype: int32

In [14]:
s.nsmallest(3)

7    0
8    1
5    2
dtype: int32

In [15]:
s.nlargest(3)

9    9
2    8
3    7
dtype: int32

从v0.17.0开始，DataFrame也有了以上两个方法。

In [16]:
df = pd.DataFrame({'a': [-2, -1, 1, 10, 8, 11, -1],
                      'b': list('abdceff'),
                       'c': [1.0, 2.0, 4.0, 3.2, np.nan, 3.0, 4.0]})

In [25]:
df

,a,b,c
0,-2,a,1.0
1,-1,b,2.0
2,1,d,4.0
3,10,c,3.2
4,8,e,NaN
5,11,f,3.0
6,-1,f,4.0


In [28]:
df.nlargest(5, 'a') #列 'a'最大的3个值

,a,b,c
5,11,f,3.0
3,10,c,3.2
4,8,e,NaN
2,1,d,4.0
1,-1,b,2.0


In [27]:
df.nlargest(5, ['a', 'c'])

,a,b,c
5,11,f,3.0
3,10,c,3.2
4,8,e,NaN
2,1,d,4.0
1,-1,b,2.0


In [22]:
df.nsmallest(3, 'a')

,a,b,c
0,-2,a,1.0
1,-1,b,2.0
6,-1,f,4.0


In [24]:
df.nsmallest(5, ['a', 'c'])

,a,b,c
0,-2,a,1.0
1,-1,b,2.0
6,-1,f,4.0
2,1,d,4.0
4,8,e,NaN


### 多索引列 排序

如果一列是多索引，你必须指定全部每一级的索引。

In [30]:
df1 = pd.DataFrame({'one':[2,1,1,1],'two':[1,3,2,4],'three':[5,4,3,2]})

In [31]:
df1.columns = pd.MultiIndex.from_tuples([('a','one'),('a','two'),('b','three')])

In [32]:
df1

a         b
  one two three
0   2   5     1
1   1   4     3
2   1   3     2
3   1   2     4

In [33]:
df1.sort_values(by=('a','two'))

a         b
  one two three
3   1   2     4
2   1   3     2
1   1   4     3
0   2   5     1

## 复制

copy()方法复制数据结构的值并返回一个新的对象。记住**复制操作不到万不得已不使用**。 比如，改变DataFrame对象值的几种方法：
* inserting, deleting, modifying a column
* 为索引、列 赋值
* 对于同构数据，直接使用values属性修改值。

**几乎所有的方法都不对原对象进行直接修改，而是返回修改后的一个新对象！如果原对象数据被修改，肯定是你显示指定的修改操作。**

 ## dtypes属性

pandas对象的主要数据类型包括: float, int, bool, datetime64[ns], datetime64[ns, tz], timedelta[ns], category, object. 除此之外还有更具体的说明存储比特数的数据类型比如int64, int32。

DataFrame的dtypes属性返回一个Series对象，Series值是DF中每一列的数据类型。

In [39]:
df = pd.DataFrame(dict(A = np.random.rand(3),
                            B = 1,
                           C = 'foo',
                            D = pd.Timestamp('20010102'),
                           E = pd.Series([1.0]*3).astype('float32'),
                                        F = False,
                                       G = pd.Series([1]*3,dtype='int8')))

In [40]:
df

,A,B,C,D,E,F,G
0,0.223911,1,foo,2001-01-02,1.0,False,1
1,0.306255,1,foo,2001-01-02,1.0,False,1
2,0.435360,1,foo,2001-01-02,1.0,False,1


In [41]:
df.dtypes

A           float64
B             int64
C            object
D    datetime64[ns]
E           float32
F              bool
G              int8
dtype: object

Series同样有dtypes属性

In [42]:
df['A'].dtype

dtype('float64')

如果pandas对象的一列中有多种数据类型，dtype返回的是能兼容所有数据类型的类型，object范围最大的。

In [43]:
pd.Series([1,2,3,4,5,6.])

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
5    6.0
dtype: float64

In [44]:
pd.Series([1,2,3,6.,'foo'])

0      1
1      2
2      3
3      6
4    foo
dtype: object

get_dtype_counts()方法返回DataFrame中每一种数据类型的列数。

In [50]:
df.get_dtype_counts()

bool              1
datetime64[ns]    1
float32           1
float64           1
int64             1
int8              1
object            1
dtype: int64

数值数据类型可以在ndarray，Series和DataFrame中传播。

In [51]:
df1 = pd.DataFrame(np.random.randn(8, 1), columns=['A'], dtype='float32')

In [52]:
df1

,A
0,0.325100
1,-1.052396
2,1.866763
3,-0.647942
4,-1.518647
5,0.982795
6,-0.068309
7,2.034926


In [53]:
df1.dtypes

A    float32
dtype: object

In [56]:
df2 = pd.DataFrame(dict( A = pd.Series(np.random.randn(8), dtype='float16'),
                         B = pd.Series(np.random.randn(8)),
                         C = pd.Series(np.array(np.random.randn(8), dtype='uint8')) )) #这里是float16， uint8
df2

,A,B,C
0,-0.427979,0.280275,0
1,-0.701172,-0.456572,0
2,-0.558105,-0.248928,255
3,-0.361572,0.746420,0
4,-1.667969,0.722280,255
5,-1.436523,-0.864915,0
6,0.147949,1.620295,0
7,0.075989,-0.527997,0


In [57]:
df2.dtypes

A    float16
B    float64
C      uint8
dtype: object

### 数据类型的默认值

整型的默认值蕾西int64，浮点型的默认类型是float64，和你用的是32位还是64位的系统无关。

In [59]:
pd.DataFrame([1, 2], columns=['a']).dtypes

a    int64
dtype: object

In [60]:
pd.DataFrame({'a': [1, 2]}).dtypes

a    int64
dtype: object

In [61]:
pd.DataFrame({'a': 1}, index=list(range(2))).dtypes

a    int64
dtype: object

Numpy中数值的具体类型则要依赖于平台。

In [64]:
frame = pd.DataFrame(np.array([1, 2])) #如果是在32位系统，数据类型int32

### upcasting

不同类型结合时会upcast，即得到更通用的类型，看例子吧：

In [75]:
df1.dtypes

A    float32
dtype: object

In [76]:
df2.dtypes

A    float16
B    float64
C      uint8
dtype: object

In [77]:
df1.reindex_like(df2).fillna(value=0.0).dtypes

A    float32
B    float64
C    float64
dtype: object

In [78]:
df3 = df1.reindex_like(df2).fillna(value=0.0) + df2

In [79]:
df3.dtypes

A    float32
B    float64
C    float64
dtype: object

### astype()方法

使用astype()显示的进行转型。默认会返回原对象的副本，即使数据类型不变。当然可以传递copy=False参数直接对原对象转型。

In [80]:
df3

,A,B,C
0,-0.102879,0.280275,0.0
1,-1.753568,-0.456572,0.0
2,1.308658,-0.248928,255.0
3,-1.009514,0.746420,0.0
4,-3.186615,0.722280,255.0
5,-0.453728,-0.864915,0.0
6,0.079640,1.620295,0.0
7,2.110915,-0.527997,0.0


In [81]:
df3.dtypes

A    float32
B    float64
C    float64
dtype: object

In [82]:
df3.astype('float32').dtypes

A    float32
B    float32
C    float32
dtype: object

### 对object类型进行转型

convert_objects()方法能对object类型进行转型。如果想转为数字，参数是convert_numeric=True。

In [84]:
df3['D'] = '1.'

In [85]:
df3['E'] = '1'

In [86]:
df3

,A,B,C,D,E
0,-0.102879,0.280275,0.0,1.,1
1,-1.753568,-0.456572,0.0,1.,1
2,1.308658,-0.248928,255.0,1.,1
3,-1.009514,0.746420,0.0,1.,1
4,-3.186615,0.722280,255.0,1.,1
5,-0.453728,-0.864915,0.0,1.,1
6,0.079640,1.620295,0.0,1.,1
7,2.110915,-0.527997,0.0,1.,1


In [88]:
df3.dtypes #现在'D' 'E'两列都是object类型

A    float32
B    float64
C    float64
D     object
E     object
dtype: object

In [89]:
df3.convert_objects(convert_numeric=True).dtypes

c:\python27\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


A    float32
B    float64
C    float64
D    float64
E      int64
dtype: object

In [90]:
df3['D'] = df3['D'].astype('float16')

In [91]:
df3['E'] = df3['E'].astype('int32')

In [92]:
df3.dtypes

A    float32
B    float64
C    float64
D    float16
E      int32
dtype: object

## 基于dtype 选择列

select_dtypes()方法实现了基于列dtype的构造子集方法。

In [93]:
df = pd.DataFrame({'string': list('abc'),
                    'int64': list(range(1, 4)),
                    'uint8': np.arange(3, 6).astype('u1'),
                    'float64': np.arange(4.0, 7.0),
                    'bool1': [True, False, True],
                    'bool2': [False, True, False],
                    'dates': pd.date_range('now', periods=3).values,
                    'category': pd.Series(list("ABC")).astype('category')})

In [94]:
df

,bool1,bool2,category,dates,float64,int64,string,uint8
0,True,False,A,2016-04-20 10:26:18.089,4.0,1,a,3
1,False,True,B,2016-04-21 10:26:18.089,5.0,2,b,4
2,True,False,C,2016-04-22 10:26:18.089,6.0,3,c,5


In [95]:
df['tdeltas'] = df.dates.diff()

In [96]:
df['uint64'] = np.arange(3, 6).astype('u8')

In [97]:
df['other_dates'] = pd.date_range('20130101', periods=3).values

In [98]:
df['tz_aware_dates'] = pd.date_range('20130101', periods=3, tz='US/Eastern')

In [99]:
df

,bool1,bool2,category,dates,float64,int64,string,uint8,tdeltas,uint64,other_dates,tz_aware_dates
0,True,False,A,2016-04-20 10:26:18.089,4.0,1,a,3,NaT,3,2013-01-01,2013-01-01 00:00:00-05:00
1,False,True,B,2016-04-21 10:26:18.089,5.0,2,b,4,1 days,4,2013-01-02,2013-01-02 00:00:00-05:00
2,True,False,C,2016-04-22 10:26:18.089,6.0,3,c,5,1 days,5,2013-01-03,2013-01-03 00:00:00-05:00


In [100]:
df.dtypes

bool1                                   bool
bool2                                   bool
category                            category
dates                         datetime64[ns]
float64                              float64
int64                                  int64
string                                object
uint8                                  uint8
tdeltas                      timedelta64[ns]
uint64                                uint64
other_dates                   datetime64[ns]
tz_aware_dates    datetime64[ns, US/Eastern]
dtype: object

select_dtypes()有两个参数：include, exclude。含义是要选择的列的dtype和不选择列的dtype。

In [104]:
df.select_dtypes(include=[bool])

,bool1,bool2
0,True,False
1,False,True
2,True,False


In [105]:
df.select_dtypes(include=['bool'])

,bool1,bool2
0,True,False
1,False,True
2,True,False


In [107]:
df.dtypes

bool1                                   bool
bool2                                   bool
category                            category
dates                         datetime64[ns]
float64                              float64
int64                                  int64
string                                object
uint8                                  uint8
tdeltas                      timedelta64[ns]
uint64                                uint64
other_dates                   datetime64[ns]
tz_aware_dates    datetime64[ns, US/Eastern]
dtype: object

In [108]:
df.select_dtypes(include=['number', 'bool'], exclude=['unsignedinteger'])

,bool1,bool2,float64,int64,tdeltas
0,True,False,4.0,1,NaT
1,False,True,5.0,2,1 days
2,True,False,6.0,3,1 days


如果要选择字符串类型的列，必须使用object类型。

In [109]:
df.select_dtypes(include=['object'])

,string
0,a
1,b
2,c


如果想要知道某种数据类型的所有子类型，比如numpy.number类型，你可以定义如下的方法：

In [110]:
def subdtypes(dtype):
        subs = dtype.__subclasses__()
        if not subs:
            return dtype
        return [dtype, [subdtypes(dt) for dt in subs]]
   

In [111]:
subdtypes(np.generic)

[numpy.generic,
 [[numpy.number,
   [[numpy.integer,
     [[numpy.signedinteger,
       [numpy.int8,
        numpy.int16,
        numpy.int32,
        numpy.int32,
        numpy.int64,
        numpy.timedelta64]],
      [numpy.unsignedinteger,
       [numpy.uint8,
        numpy.uint16,
        numpy.uint32,
        numpy.uint32,
        numpy.uint64]]]],
    [numpy.inexact,
     [[numpy.floating,
       [numpy.float16, numpy.float32, numpy.float64, numpy.float64]],
      [numpy.complexfloating,
       [numpy.complex64, numpy.complex128, numpy.complex128]]]]]],
  [numpy.flexible,
   [[numpy.character, [numpy.string_, numpy.unicode_]],
    [numpy.void, [numpy.record]]]],
  numpy.bool_,
  numpy.datetime64,
  numpy.object_]]